## Create Corpus file of Offitial training data
* Remove non-chinese-word
* Use jieba cut
* 輸出到 'datas/official_all_corpus.txt'

In [1]:
# Set up tokenize dictionary
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

Building prefix dict from /home/sunset/word_contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u849ecfdca27003d306f39ca004b82b5b.cache
Loading model cost 1.174 seconds.
Prefix dict has been built succesfully.


* 在 'datas/training_data/subtitle_no_TC/我的這一班/聽稿版' 裡的 files 有包含說話者人名，先不要用

In [2]:
import json
import os
import pdb 
import re

import jieba

not_chinese_word = u'[^\u4e00-\u9fff]'  # only keep Chinese word, remove 

def create_corpus(out_filename, file_mode='w'):
    out_f = open(out_filename, file_mode)
    
    all_corpus_dirs = ['datas/training_data/subtitle_with_TC', 'datas/training_data/subtitle_no_TC']
    for corpus_dir in all_corpus_dirs:
        for _, dirs, corpus_files in os.walk(corpus_dir):
            for dir in dirs:
                for _,_,files in os.walk(corpus_dir+'/'+dir):
                    for fn in files:
                        if fn[0] == '.':
                            continue
                        try:
                            f = open(corpus_dir+'/'+dir+'/'+fn, 'r')
                            for line in f:
                                line = re.sub(not_chinese_word, ' ', line)

                                # word segmentation with jieba.
                                words = list(jieba.cut(line))
                                #words = [w for w in words if w != ' ']
                                # write to file.
                                out_f.write('%s\n' % ' '.join(words))
                        except:
                            print('no such file: %s' %corpus_dir+'/'+dir+'/'+fn, end='\t')
                            

if __name__ == '__main__':
    create_corpus('datas/official_all_corpus.txt', 'w')

no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(460)_G000021604600005_010.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(461)_G000021604610005_002.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(464)_G000021604640005_008.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(472)_G000021604720005_024.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(465)_G000021604650005_010.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(462)_G000021604620005_004.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(466)_G000021604660005_012.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(467)_G000021604670005_014.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(471)_G000021604710009_022.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(470)_G000021604700006_020.txt	no such file: datas/training_data/subtitle_no_TC/我的這一班/我的這一班(473)_G000021604730006_026.txt

## Train Word2vec

In [25]:
from gensim.models import word2vec

# Create sentences from preprocessed corpus file
sentences = word2vec.LineSentence('datas/official_all_corpus.txt')

# Show tokenize "sentence"
for i, s in enumerate(sentences):
    print()
    print(s)
    print('=' * 100)
    if i >= 1:
        break


['躲', '推銷員', '和', '金蟬脫殼']

['有', '什麼', '關係', '呢']


#### 助教說是 train word2vec 不錯的參數:
-./word2vec/trunk/word2vec -train corpus.txt -output my.cbow.200d.txt -size 200 -windows 5 sample 1e-4 -negative 10 -hs 0 -cbow 1 -iter 15 -threads 8 -min - count 5

In [31]:
# Train word2vec model
# word2vec_model = word2vec.Word2Vec(sentences, size=200, window=5, workers=4, min_count=5, sample=1e-4, negative=10, iter=15)
# word2vec_model.total_train_time

# word2vec_model.save('models/word2vec_all_offitial_200.model.bin')

389.8865989702754

In [ ]:
# Retrain
# word2vec_model = gensim.models.Word2Vec.load('models/word2vec_all_offitial_200.model.bin')
# word2vec_model.train(sentences)

## 由現有 word2vec model 建立 dictionary

In [37]:
from gensim.models import word2vec

word2vec_model = word2vec.Word2Vec.load('models/word2vec_all_offitial_200.model.bin')

In [40]:
vocab = word2vec_model.wv.vocab
print('朋友 -> ', word2vec_model.wv.vocab['朋友'])
print('朋友 -> ', word2vec_model.wv.vocab['朋友'].index)

朋友 ->  Vocab(count:18080, index:100, sample_int:1483476828)
朋友 ->  100


## 將公視文字資料轉成 word2vec model 對應的 vocab id
* 存成 npy

## 輸出 embedding 
* [輸出給 keras 用](http://ben.bolte.cc/blog/2016/gensim.html)